# Generation of tables and figures of MRIQC paper

This notebook is associated to the paper:

Esteban O, Birman D, Schaer M, Koyejo OO, Poldrack RA, Gorgolewski KJ; MRIQC: Predicting Quality in Manual MRI Assessment Protocols Using No-Reference Image Quality Measures; bioRxiv 111294; doi:[10.1101/111294](https://doi.org/10.1101/111294).

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os.path as op
import numpy as np
import pandas as pd
from mriqc.viz import misc as mviz
from pkg_resources import resource_filename as pkgrf

outputs_path = '../../mriqc-data/'
abide_path = '/home/oesteban/Data/ABIDE/'

## Figure 1: artifacts in MRI
Shows a couple of subpar datasets from the ABIDE dataset

In [ ]:
mviz.figure1(
    op.join(abide_path, 'sub-50137', 'anat', 'sub-50137_T1w.nii.gz'),
    op.join(abide_path, 'sub-50110', 'anat', 'sub-50110_T1w.nii.gz'),
    op.join(outputs_path, 'figures', 'fig01-artifacts.pdf'))

## Other figures: read some data

In [ ]:
x_path = pkgrf('mriqc', 'data/csv/x_abide-0.9.6-2017-06-03-99db97c9be2e.csv')
y_path = pkgrf('mriqc', 'data/csv/y_abide.csv')
ds030_x_path = pkgrf('mriqc', 'data/csv/x_ds030-0.9.6-2017-06-03-99db97c9be2e.csv')
ds030_y_path = pkgrf('mriqc', 'data/csv/y_ds030.csv')

rater_types = {'rater_1': float, 'rater_2': float, 'rater_3': float}
mdata = pd.read_csv(y_path, index_col=False, dtype=rater_types)

sites = list(sorted(list(set(mdata.site.values.ravel().tolist()))))

## Inter-rater variability

In [ ]:
from sklearn.metrics import cohen_kappa_score
overlap = mdata[np.all(~np.isnan(mdata[['rater_1', 'rater_2']]), axis=1)]
y1 = overlap.rater_1.values.ravel().tolist()
y2 = overlap.rater_2.values.ravel().tolist()
fig = mviz.inter_rater_variability(y1, y2, out_file=op.join(outputs_path, 'figures', 'fig02-irv.pdf'))

print("Cohen's Kappa %f" % cohen_kappa_score(y1, y2))

y1 = overlap.rater_1.values.ravel()
y1[y1 == 0] = 1

y2 = overlap.rater_2.values.ravel()
y2[y2 == 0] = 1
print("Cohen's Kappa (binarized): %f" % cohen_kappa_score(y1, y2))


In [ ]:
rfc_roc_auc=[0.597, 0.380, 0.857, 0.610, 0.698, 0.692, 0.963, 0.898, 0.772, 0.596, 0.873, 0.729, 0.784, 0.860, 0.751, 0.900, 0.489]
rfc_acc=[0.842, 0.815, 0.648, 0.609, 0.789, 0.761, 0.893, 0.833, 0.842, 0.767, 0.806, 0.850, 0.878, 0.798, 0.559, 0.881, 0.375]
svc_lin_roc_auc=[0.583, 0.304, 0.943, 0.668, 0.691, 0.754, 1.000, 0.778, 0.847, 0.590, 0.857, 0.604, 0.604, 0.838, 0.447, 0.650, 0.501]
svc_lin_acc=[0.947, 0.667, 0.870, 0.734, 0.754, 0.701, 0.750, 0.639, 0.877, 0.767, 0.500, 0.475, 0.837, 0.768, 0.717, 0.050, 0.429]
svc_rbf_roc_auc=[0.681, 0.217, 0.827, 0.553, 0.738, 0.616, 0.889, 0.813, 0.845, 0.658, 0.779, 0.493, 0.726, 0.510, 0.544, 0.500, 0.447]
svc_rbf_acc=[0.947, 0.852, 0.500, 0.578, 0.772, 0.712, 0.821, 0.583, 0.912, 0.767, 0.500, 0.450, 0.837, 0.778, 0.441, 0.950, 0.339]

df = pd.DataFrame({
    'site': list(range(len(sites))) * 3,
    'accuracy': rfc_acc + svc_lin_acc + svc_rbf_acc,
    'model': ['RFC'] * len(sites) + ['SVC_lin'] * len(sites) + ['SVC_rbf'] * len(sites)
    
})


x = np.arange(len(sites))
data = list(zip(rfc_roc_auc, svc_lin_roc_auc, svc_rbf_roc_auc))

dim = len(data[0])
w = 0.75
dimw = w / dim

for i in range(dim):
    y = [d[i] for d in data]
    b = plt.bar(x + i * dimw, y, dimw, bottom=0.001)

# plt.plot(rfc_roc_auc, marker='o', markersize=10, fillstyle='none')
# plt.axhline(np.average(rfc_roc_auc))
# plt.plot(svc_lin_roc_auc, marker='o', lw=.5)
# plt.axhline(np.average(svc_lin_roc_auc))
# plt.plot(svc_rbf_roc_auc, marker='o', lw=.5)
# plt.axhline(np.average(svc_rbf_roc_auc))
# plt.xticks(range(len(sites)), sites, rotation='vertical')


## Confusion matrix on DS030

In [ ]:
from sklearn.metrics import confusion_matrix

pred_file = op.abspath(op.join(
    '..', 'mriqc/data/csv',
    'mclf_run-20170703-190702_mod-rfc_ver-0.9.7.clf-3.3_class-2_cv-loso_data-test_pred.csv'))

pred_y = pd.read_csv(pred_file)
true_y = pd.read_csv(ds030_y_path)
true_y.rater_1 *= -1
true_y.rater_1[true_y.rater_1 < 0] = 0
print(confusion_matrix(true_y.rater_1.tolist(), pred_y.pred_y.values.ravel().tolist(), labels=[0, 1]))

## Feature importances

In [ ]:
from sklearn.externals.joblib import load as loadpkl
estimator = loadpkl(pkgrf('mriqc', 'data/mclf_run-20170703-190702_mod-rfc_ver-0.9.7.clf-3.3_class-2_cv-loso_data-train_estimator.pklz'))

forest = estimator.named_steps['rfc']
features = ["cjv", "cnr", "efc", "fber", "fwhm_avg", "fwhm_x", "fwhm_y", "fwhm_z", "icvs_csf", "icvs_gm", "icvs_wm", "qi_1", "qi_2", "rpve_csf", "rpve_gm", "rpve_wm", "snr_csf", "snr_gm", "snr_total", "snr_wm", "snrd_csf", "snrd_gm", "snrd_total", "snrd_wm", "summary_bg_k", "summary_bg_stdv", "summary_csf_k", "summary_csf_mad", "summary_csf_mean", "summary_csf_median", "summary_csf_p05", "summary_csf_p95", "summary_csf_stdv", "summary_gm_k", "summary_gm_mad", "summary_gm_mean", "summary_gm_median", "summary_gm_p05", "summary_gm_p95", "summary_gm_stdv", "summary_wm_k", "summary_wm_mad", "summary_wm_mean", "summary_wm_median", "summary_wm_p05", "summary_wm_p95", "summary_wm_stdv", "tpm_overlap_csf", "tpm_overlap_gm", "tpm_overlap_wm"]

In [ ]:
import seaborn as sn
sn.set_style("white")
nft = len(features)

forest = estimator.named_steps['rfc']
importances = np.median([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
# importances = np.median(, axis=0)
indices = np.argsort(importances)[::-1]

df = {'Feature': [], 'Importance': []}
for tree in forest.estimators_:
    for i in indices:
        df['Feature'] += [features[i]]
        df['Importance'] += [tree.feature_importances_[i]]
fig = plt.figure(figsize=(20, 6))
# plt.title("Feature importance plot")
sn.boxplot(x='Feature', y='Importance', data=pd.DataFrame(df), linewidth=1, notch=True)
plt.xlabel('Features selected (%d)' % len(features))
# plt.bar(range(nft), importances[indices],
#        color="r", yerr=std[indices], align="center")
plt.xticks(range(nft))
plt.gca().set_xticklabels([features[i] for i in indices], rotation=90)
plt.xlim([-1, nft])
plt.show()
fig.savefig(op.join(outputs_path, 'figures', 'fig06-exp2-fi.pdf'),
            bbox_inches='tight', pad_inches=0, dpi=300)